# This is a juypter notebook classifing gestures based on sEMG-Signals. 
The Data is aquiered from: https://archive.ics.uci.edu/ml/datasets/sEMG+for+Basic+Hand+movements


In [142]:
import numpy as np 
import pandas as pd
import scipy.io


In [143]:
mat = scipy.io.loadmat('sEMG_data/Database 1/female_1.mat')

In [144]:
print(mat["cyl_ch2"].shape)
mat.keys()

(30, 3000)


dict_keys(['__header__', '__version__', '__globals__', 'cyl_ch1', 'cyl_ch2', 'hook_ch1', 'hook_ch2', 'tip_ch1', 'tip_ch2', 'palm_ch1', 'palm_ch2', 'spher_ch1', 'spher_ch2', 'lat_ch1', 'lat_ch2'])

# Make Numpy array from MATLAB Structured array

As there are two sensors the data aquired by each sensor is depth stacked, while the time is horizontal stacked and the gestures are vertical stacked 

In [145]:
fs_items, ss_items = [],[]
output = []
for fsensor, ssensor in list(zip(list(mat.keys())[3:][::2], list(mat.keys())[3:][1::2])):
    fs_items.append(mat[fsensor])
    ss_items.append(mat[ssensor])
    output.extend([fsensor.split("_")[0]] * len(mat[fsensor]))
arr = np.dstack([np.vstack(fs_items), np.vstack(ss_items)])
output = np.vstack(output)

In [146]:
print(arr.shape)

(180, 3000, 2)


In [147]:
print(output.shape)

(180, 1)


### One-Hot-Encoding

The outcome (gestures) shall be one-hot encoded

In [148]:
from keras.utils import to_categorical

str_obj, encoded_obj = np.unique(output, return_inverse=True)
encoded_obj = np.sort(encoded_obj)
str_obj, encoded_obj

(array(['cyl', 'hook', 'lat', 'palm', 'spher', 'tip'], dtype='<U5'),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5]))

In [149]:
output = to_categorical(encoded_obj)

print(*["{} belogs to {} \n".format(x, y) for x, y in zip(str_obj, np.unique(output, axis=0))])

cyl belogs to [0. 0. 0. 0. 0. 1.] 
 hook belogs to [0. 0. 0. 0. 1. 0.] 
 lat belogs to [0. 0. 0. 1. 0. 0.] 
 palm belogs to [0. 0. 1. 0. 0. 0.] 
 spher belogs to [0. 1. 0. 0. 0. 0.] 
 tip belogs to [1. 0. 0. 0. 0. 0.] 



### Sliding windows on the data 

The data is sampled at a sample rate of 500Hz. 3000 datapoints lead to a recording time of 6s. 
Every sliding window shall be 200ms long with a 50% overlap. Leading to: 200/1000 * 500 = 100 values per window

In [150]:
from numpy.lib.stride_tricks import sliding_window_view
x = np.arange(12)
v = sliding_window_view(x, 4)[::2]
v

array([[ 0,  1,  2,  3],
       [ 2,  3,  4,  5],
       [ 4,  5,  6,  7],
       [ 6,  7,  8,  9],
       [ 8,  9, 10, 11]])

##### Sliding Window for one row

In [151]:
v = np.mean(sliding_window_view(arr[0, :, 0], 200)[::100], axis=1)
v

array([0.14526026, 0.14424018, 0.14819295, 0.14500529, 0.14207263,
       0.13748232, 0.13238199, 0.13161696, 0.13416714, 0.1447503 ,
       0.13799236, 0.1391399 , 0.14105252, 0.14105256, 0.13799237,
       0.13085188, 0.12562403, 0.14538781, 0.14883053, 0.13148941,
       0.1452603 , 0.15048814, 0.15036063, 0.1436027 , 0.14870302,
       0.14041495, 0.13607967, 0.14462274, 0.14169005])

##### Sliding window for 2d-Array 

In [152]:
a = np.arange(28).reshape(4, 7)
a

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27]])

In [153]:
a.shape

(4, 7)

In [154]:
v = sliding_window_view(a, (4, 3))[:, ::2]
v

array([[[[ 0,  1,  2],
         [ 7,  8,  9],
         [14, 15, 16],
         [21, 22, 23]],

        [[ 2,  3,  4],
         [ 9, 10, 11],
         [16, 17, 18],
         [23, 24, 25]],

        [[ 4,  5,  6],
         [11, 12, 13],
         [18, 19, 20],
         [25, 26, 27]]]])

In [155]:
arr.shape

(180, 3000, 2)

In [156]:
v = sliding_window_view(arr[:, :, 0], window_shape=(180, 200))[:, ::50]

In [157]:
window_arr = np.mean(v, axis=3)[0].T
window_arr[0, 0]

0.14526025500000006

In [158]:
np.sum(arr[0, :200, 0])/200

0.145260255

### Sliding windows on 3D-Array 

I tried applying this to 3D-Array but it did'nt work. Use for-loop instead

In [159]:
arr[28:, :, 0]

array([[-0.947867,  1.117765,  1.959319, ...,  0.225208,  0.735241,
         1.321778],
       [ 0.148703,  0.148703,  0.454723, ...,  1.015759,  0.735241,
        -0.106313],
       [ 0.225208,  0.352716,  0.301713, ..., -0.386831,  0.148703,
         0.735241],
       ...,
       [ 0.0977  , -0.106313, -0.004307, ...,  0.174205,  0.072198,
        -0.029808],
       [ 0.199706,  0.327214,  0.123201, ...,  0.199706,  0.429221,
         0.199706],
       [ 0.174205,  0.454723,  0.072198, ...,  0.046696,  0.046696,
         0.199706]])

In [160]:
def apply_sliding_windows(array, window_size=200, overlap=None): 
    if overlap is None: 
        overlap = int(0.25 * window_size)
    windows = sliding_window_view(array, window_shape=(array.shape[0], window_size))[:, ::overlap]
    return np.mean(windows, axis=3)[0].T

In [161]:
# check 
arr.shape
np.sum(arr[0, 100:300, 0]) / 200 

0.14424018

In [162]:
apply_sliding_windows(arr[:, :, 0])[0, 2]

0.14424018

In [163]:
a = np.arange(28).reshape(4, 7)
apply_sliding_windows(a, 4)

array([[ 1.5,  2.5,  3.5,  4.5],
       [ 8.5,  9.5, 10.5, 11.5],
       [15.5, 16.5, 17.5, 18.5],
       [22.5, 23.5, 24.5, 25.5]])

In [164]:
b = np.arange(28).reshape(4, 7)
c = np.arange(28, 56).reshape(4, 7)
d = np.dstack([b, c])
d[:, :, 0]

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27]])

In [165]:
d[:, :, 1]

array([[28, 29, 30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39, 40, 41],
       [42, 43, 44, 45, 46, 47, 48],
       [49, 50, 51, 52, 53, 54, 55]])

In [166]:
w = sliding_window_view(d, window_shape=(4, 3, 1))[0, ::2, :, :, :, 0]

In [167]:
w.shape

(3, 2, 4, 3)

In [168]:
z = np.transpose(np.mean(w, axis=3), (2, 0, 1))

In [169]:
z

array([[[ 1., 29.],
        [ 3., 31.],
        [ 5., 33.]],

       [[ 8., 36.],
        [10., 38.],
        [12., 40.]],

       [[15., 43.],
        [17., 45.],
        [19., 47.]],

       [[22., 50.],
        [24., 52.],
        [26., 54.]]])

In [170]:
z.shape

(4, 3, 2)

In [171]:
z[:, : , 0]

array([[ 1.,  3.,  5.],
       [ 8., 10., 12.],
       [15., 17., 19.],
       [22., 24., 26.]])

In [172]:
def apply_sliding_windows_3axis(array, window_size=200, overlap=None):
    if overlap is None: 
        overlap = int(0.25 * window_size)
    w = sliding_window_view(array, (array.shape[0], window_size, 1))[0, ::overlap, :, :, :, 0]
    return np.transpose(np.mean(w, axis=3), (2, 0, 1))

In [173]:
arr = apply_sliding_windows_3axis(arr)
arr.shape

(180, 57, 2)

### Seperate Train an test data

In [174]:
output.shape

(180, 6)

In [175]:
arr.shape

(180, 57, 2)

In [176]:
def split_data(x, y, test_size=0.33): 
    """ Returns the data splitted into train and test data on the row"""
    assert len(x) == len(y)
    permutation = np.random.RandomState(seed=42).permutation(len(x))
    return x[permutation][int(test_size * len(x)):], x[permutation][:int(test_size * len(x))], y[permutation][int(test_size * len(x)):], y[permutation][:int(test_size * len(x))]

In [177]:
x_train, x_test, y_train, y_test = split_data(arr, output, 0.2)
x_train[0, :, 0], y_train[0]

(array([0.14181756, 0.14704539, 0.1457703 , 0.14143504, 0.140925  ,
        0.14054247, 0.13990493, 0.1453878 , 0.1422001 , 0.14271012,
        0.14309266, 0.13977743, 0.14041496, 0.13786482, 0.13607968,
        0.13811983, 0.14092502, 0.13964993, 0.14156257, 0.14003245,
        0.13722726, 0.14475024, 0.13901237, 0.14041497, 0.14079749,
        0.13646221, 0.14207256, 0.14143503, 0.14194509, 0.1434752 ,
        0.14105256, 0.14347518, 0.14334766, 0.13977743, 0.14207257,
        0.13901239, 0.1407975 , 0.14156254, 0.1395224 , 0.13875737,
        0.13773729, 0.13837483, 0.13964991, 0.13862982, 0.13684472,
        0.14066995, 0.14169003, 0.14385767, 0.14577028, 0.14245508,
        0.1441127 , 0.14220009, 0.14015998, 0.14130756, 0.14105256,
        0.14258264, 0.14283766]),
 array([0., 0., 0., 1., 0., 0.], dtype=float32))

In [178]:
x_test[0, :, 0], y_test[0]

(array([0.14143503, 0.13034184, 0.13735479, 0.13786485, 0.13646224,
        0.14730042, 0.1346771 , 0.14781044, 0.15507843, 0.14755544,
        0.1530383 , 0.14245513, 0.12906677, 0.14028751, 0.14041503,
        0.13276455, 0.1495956 , 0.13862989, 0.14334768, 0.15890368,
        0.14768298, 0.14870305, 0.14360273, 0.13595222, 0.12753666,
        0.14322015, 0.14169004, 0.12944929, 0.13773733, 0.13607973,
        0.13429463, 0.1493406 , 0.14985064, 0.1351872 , 0.14526034,
        0.13786485, 0.13990495, 0.15227326, 0.15036064, 0.14003249,
        0.13735482, 0.14360269, 0.13569718, 0.15265574, 0.15941368,
        0.15112566, 0.15431338, 0.15163571, 0.15278331, 0.14003251,
        0.1275367 , 0.14449529, 0.13467714, 0.14934057, 0.15265578,
        0.14615287, 0.14844802]),
 array([1., 0., 0., 0., 0., 0.], dtype=float32))

### Train the lstm model

In [179]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense


In [180]:
epochs, batch_size = 15, 64
verbose, n_steps, n_length = 0, 4, 32
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]

input_shape = (n_timesteps, n_features)

In [181]:
model = Sequential()
model.add(LSTM(57, input_shape=input_shape))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dense(n_outputs, activation="softmax"))

In [194]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

In [195]:
x_test.shape

(36, 57, 2)

In [196]:
y_test.shape

(36, 6)

In [197]:
_, accuracy = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=verbose)

In [198]:
'Accuracy: {:.4f}'.format(accuracy)

'Accuracy: 0.1806'

In [193]:
y_test

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0.